In [74]:
%matplotlib inline

In [75]:
!pip install torchmetrics

Download the dataset

In [76]:
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip -o fra-eng.zip
!mkdir data
!mv fra.txt data/eng-fra.txt

--2025-04-04 04:17:00--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip.1’

fra-eng.zip.1       100%[===================>]   7.57M  5.72MB/s    in 1.3s    

2025-04-04 04:17:02 (5.72 MB/s) - ‘fra-eng.zip.1’ saved [7943074/7943074]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 
mkdir: cannot create directory ‘data’: File exists


In [77]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading data files
==================

The data for this project is a set of many thousands of English to
French translation pairs.

`This question on Open Data Stack
Exchange <http://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages>`__
pointed me to the open translation site http://tatoeba.org/ which has
downloads available at http://tatoeba.org/eng/downloads - and better
yet, someone did the extra work of splitting language pairs into
individual text files here: http://www.manythings.org/anki/

The English to French pairs are too big to include in the repo, so
download to ``data/eng-fra.txt`` before continuing. The file is a tab
separated list of translation pairs:

::

    I am cold.    J'ai froid.

.. Note::
   Download the data from
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   and extract it to the current directory.

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.


In [78]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.

In [79]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we're filtering to sentences that translate to
the form "I am" or "He is" etc. (accounting for apostrophes replaced
earlier).

In [80]:
MAX_LENGTH = 15

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re"
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs

In [81]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
#print(random.choice(pairs))

Reading lines...
Read 232736 sentence pairs
Trimmed to 22907 sentence pairs
Counting words...
Counted words:
fra 7019
eng 4638


In [82]:
from sklearn.model_selection import train_test_split

In [83]:
X = [i[0] for i in pairs]
y = [i[1] for i in pairs]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [85]:
train_pairs = list(zip(X_train,y_train))
test_pairs = list(zip(X_test,y_test))

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

In [86]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.



Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).

In [87]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        # Simple decoder implementation:
        # - input: current token (initially the SOS token)
        # - hidden: current hidden state (initially the encoder's context vector)
        # 1. Embed the input token
        output = self.embedding(input).view(1, 1, -1)
        # 2. Apply non-linearity
        output = F.relu(output)
        # 3. Run GRU for one step, using the current hidden state
        output, hidden = self.gru(output, hidden)
        # 4. Transform GRU output to vocabulary size and apply log softmax
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.

In [88]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [89]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.

In [90]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.

In [91]:
def trainIters(encoder, decoder, epochs, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    iter = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print("Epoch: %d/%d" % (epoch, epochs))
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)

            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            iter +=1

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.

In [93]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:

In [94]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [95]:
from torchmetrics.text.rouge import ROUGEScore
from tqdm import tqdm
import numpy as np

rouge = ROUGEScore()

def test(encoder, decoder, testing_pairs):
    input = []
    gt = []
    predict = []
    metric_score = {
        "rouge1_fmeasure":[],
        "rouge1_precision":[],
        "rouge1_recall":[],
        "rouge2_fmeasure":[],
        "rouge2_precision":[],
        "rouge2_recall":[]
    }
    from tqdm import tqdm
    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)

        input.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

        try:
            rs = rouge(output_sentence, pair[1])
        except:
            continue
        metric_score["rouge1_fmeasure"].append(rs['rouge1_fmeasure'])
        metric_score["rouge1_precision"].append(rs['rouge1_precision'])
        metric_score["rouge1_recall"].append(rs['rouge1_recall'])
        metric_score["rouge2_fmeasure"].append(rs['rouge2_fmeasure'])
        metric_score["rouge2_precision"].append(rs['rouge2_precision'])
        metric_score["rouge2_recall"].append(rs['rouge2_recall'])

    metric_score["rouge1_fmeasure"] = np.array(metric_score["rouge1_fmeasure"]).mean()
    metric_score["rouge1_precision"] = np.array(metric_score["rouge1_precision"]).mean()
    metric_score["rouge1_recall"] = np.array(metric_score["rouge1_recall"]).mean()
    metric_score["rouge2_fmeasure"] = np.array(metric_score["rouge2_fmeasure"]).mean()
    metric_score["rouge2_precision"] = np.array(metric_score["rouge2_precision"]).mean()
    metric_score["rouge2_recall"] = np.array(metric_score["rouge2_recall"]).mean()

    print("=== Evaluation score - Rouge score ===")
    print("Rouge1 fmeasure:\t",metric_score["rouge1_fmeasure"])
    print("Rouge1 precision:\t",metric_score["rouge1_precision"])
    print("Rouge1 recall:  \t",metric_score["rouge1_recall"])
    print("Rouge2 fmeasure:\t",metric_score["rouge2_fmeasure"])
    print("Rouge2 precision:\t",metric_score["rouge2_precision"])
    print("Rouge2 recall:  \t",metric_score["rouge2_recall"])
    print("=====================================")
    return input,gt,predict,metric_score

Task 1: Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = Decoder(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 5, print_every=5000)

Epoch: 0/5
8m 6s (- 158m 58s) (5000 4%) 3.3341
16m 49s (- 156m 31s) (10000 9%) 2.8450
25m 22s (- 149m 2s) (15000 14%) 2.5722
34m 2s (- 141m 25s) (20000 19%) 2.3923
Epoch: 1/5
42m 33s (- 132m 55s) (25000 24%) 2.2279
50m 58s (- 124m 11s) (30000 29%) 2.0778
59m 26s (- 115m 37s) (35000 33%) 1.9412
67m 59s (- 107m 12s) (40000 38%) 1.8603
Epoch: 2/5
76m 41s (- 98m 59s) (45000 43%) 1.7719
85m 40s (- 90m 56s) (50000 48%) 1.6602
94m 30s (- 82m 37s) (55000 53%) 1.5854
103m 29s (- 74m 18s) (60000 58%) 1.5194
Epoch: 3/5
112m 29s (- 65m 53s) (65000 63%) 1.4550
121m 41s (- 57m 30s) (70000 67%) 1.3896
130m 49s (- 48m 58s) (75000 72%) 1.3315
139m 48s (- 40m 20s) (80000 77%) 1.2722
Epoch: 4/5
148m 59s (- 31m 41s) (85000 82%) 1.2223
158m 6s (- 22m 58s) (90000 87%) 1.1786
167m 8s (- 14m 12s) (95000 92%) 1.1302
176m 12s (- 5m 25s) (100000 97%) 1.0757


In [ ]:
evaluateRandomly(encoder1, decoder1)

> je suis vraiment malheureux a ce sujet .
= i m really unhappy about this .
< i m really sad about this . <EOS>

> je suis un bon cuisinier .
= i m a good cook .
< i m a good cook . <EOS>

> je ne suis pas extraverti .
= i m not outgoing .
< i m not finicky . <EOS>

> elle porte une bague de prix .
= she is wearing a valuable ring .
< she is wearing a strong . . <EOS>

> vous conduisez horriblement mal .
= you re a horrible driver .
< you re embarrassing trouble . <EOS>

> vous etes en securite ici .
= you re safe here .
< you re safe here . <EOS>

> on va faire tout ce qu on peut .
= we re going to do everything we can .
< we re going to do everything we can . <EOS>

> j en ai termine .
= i m finished .
< i m done . <EOS>

> je suis amoureux d elle .
= i m in love with her .
< i am in love with her . <EOS>

> tu es fatiguee .
= you re tired .
< you re tired . <EOS>



In [ ]:
input,gt,predict,score = test(encoder1, decoder1, train_pairs)

100%|██████████| 20616/20616 [7:20:25<00:00,  1.28s/it]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.7188376
Rouge1 precision:	 0.67666286
Rouge1 recall:  	 0.7743015
Rouge2 fmeasure:	 0.5675114
Rouge2 precision:	 0.52466303
Rouge2 recall:  	 0.62704736


In [ ]:
input,gt,predict,score = test(encoder1, decoder1, test_pairs)

100%|██████████| 2291/2291 [1:37:40<00:00,  2.56s/it]

=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.62125415
Rouge1 precision:	 0.58754724
Rouge1 recall:  	 0.66836804
Rouge2 fmeasure:	 0.43808696
Rouge2 precision:	 0.40724462
Rouge2 recall:  	 0.4832314


Task 2: Replace GRU with LSTM (Encoder and Decoder)
=======================
Change the GRU in Encoder and Decoder in the original code base with LSTM, run the code, and record the Rouge scores for test set

In [23]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# Initialize the models
hidden_size = 256
encoder_lstm = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoder_lstm = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

Train the model



In [25]:
# Train the models
trainIters(encoder_lstm, decoder_lstm, 5, print_every=5000)

Epoch: 0/5
1m 22s (- 27m 7s) (5000 4%) 3.4136
2m 48s (- 26m 6s) (10000 9%) 3.0247
4m 10s (- 24m 33s) (15000 14%) 2.8142
5m 33s (- 23m 5s) (20000 19%) 2.6374
Epoch: 1/5
6m 57s (- 21m 45s) (25000 24%) 2.4688
8m 22s (- 20m 25s) (30000 29%) 2.3147
9m 49s (- 19m 6s) (35000 33%) 2.1873
11m 14s (- 17m 43s) (40000 38%) 2.1031
Epoch: 2/5
12m 40s (- 16m 21s) (45000 43%) 2.0131
14m 5s (- 14m 57s) (50000 48%) 1.8935
15m 31s (- 13m 34s) (55000 53%) 1.8073
16m 56s (- 12m 9s) (60000 58%) 1.7653
Epoch: 3/5
18m 20s (- 10m 44s) (65000 63%) 1.6814
19m 44s (- 9m 19s) (70000 67%) 1.6167
21m 8s (- 7m 54s) (75000 72%) 1.5558
22m 32s (- 6m 30s) (80000 77%) 1.5093
Epoch: 4/5
23m 57s (- 5m 5s) (85000 82%) 1.4483
25m 21s (- 3m 41s) (90000 87%) 1.3916
26m 45s (- 2m 16s) (95000 92%) 1.3358
28m 9s (- 0m 52s) (100000 97%) 1.2986


Save the model

In [27]:
# Save the model
torch.save(encoder_lstm.state_dict(), 'encoder_lstm.pth')
torch.save(decoder_lstm.state_dict(), 'decoder_lstm.pth')

Evaluate randomly

In [28]:
# Evaluate randomly
evaluateRandomly(encoder_lstm, decoder_lstm)

> je suis un electricien .
= i m an electrician .
< i m an electrician . <EOS>

> il est connu en tant que chanteur de rock .
= he is known as a rock singer .
< he is known as a lot of her . <EOS>

> il crie beaucoup .
= he shouts a lot .
< he s very much . <EOS>

> je ne suis pas un menteur .
= i m no liar .
< i m not a liar . <EOS>

> ils refuserent de songer a partir .
= they refused to think of leaving .
< they re to to about her . <EOS>

> nous nous deshabillons .
= we re undressing .
< we re leaving . <EOS>

> ce sont tous les deux de bons professeurs .
= they are both good teachers .
< they re both good love they re . <EOS>

> je suis desole si je vous ai derangee .
= i m sorry if i disturbed you .
< i m sorry if i embarrassed you . <EOS>

> je suis dans la meme galere .
= i m in the same boat .
< i m in the same boat . <EOS>

> vous etes tres flexibles .
= you re very flexible .
< you re very sophisticated . <EOS>



Test with test pairs and record Rouge scores

In [29]:
# Test with test pairs and record Rouge scores
input_lstm, gt_lstm, predict_lstm, score_lstm = test(encoder_lstm, decoder_lstm, test_pairs)

100%|██████████| 2291/2291 [03:48<00:00, 10.02it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.61341244
Rouge1 precision:	 0.578206
Rouge1 recall:  	 0.66123617
Rouge2 fmeasure:	 0.43128422
Rouge2 precision:	 0.3999701
Rouge2 recall:  	 0.4761495


Task 3: Replace Encoder GRU with Bi-LSTM (Decoder remains GRU)
=======================
Change the GRU in Encoder (not Decoder) in the original code base with bi-LSTM, run the code, and record the Rouge scores for test set

Change the training function

In [96]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # Adjust encoder outputs size based on encoder type
    output_size = encoder.hidden_size
    if isinstance(encoder, EncoderBiLSTM):
        output_size = encoder.hidden_size * 2  # Double size for bidirectional

    # FIX: Use square brackets instead of parentheses for shape arguments
    encoder_outputs = torch.zeros([max_length, output_size], device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        # Handle different encoder types
        if isinstance(encoder, EncoderBiLSTM):
            # For BiLSTM - output[0] already contains concatenated directions
            encoder_outputs[ei] = encoder_output[0]
        else:
            # For regular encoders
            encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    # If using BiLSTM, we need to prepare hidden state for the decoder
    if isinstance(encoder, EncoderBiLSTM):
        # Take only forward direction (or average both directions)
        h = encoder_hidden[0][0].unsqueeze(0)  # Take first direction's hidden state
        decoder_hidden = h
    else:
        decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Change the evaluate function

In [97]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # ...
        # Adjust encoder outputs size based on encoder type
        output_size = encoder.hidden_size
        if isinstance(encoder, EncoderBiLSTM):
            output_size = encoder.hidden_size * 2  # Double size for bidirectional

        # FIX: Pass dimensions as a tuple rather than separate arguments
        encoder_outputs = torch.zeros((max_length, output_size), device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

            # Handle different encoder types
            if isinstance(encoder, EncoderBiLSTM):
                encoder_outputs[ei] = encoder_output[0]
            else:
                encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        # If using BiLSTM, prepare hidden state for the decoder
        if isinstance(encoder, EncoderBiLSTM):
            h = encoder_hidden[0][0].unsqueeze(0)  # Take first direction's hidden state
            decoder_hidden = h
        else:
            decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

Create the encoder

In [98]:
class EncoderBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderBiLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        # Bidirectional LSTM - setting num_layers explicitly to 1
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=1, bidirectional=True)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        # Pass the embedded input and hidden state through the BiLSTM
        output, hidden = self.lstm(embedded, hidden)
        # Return output and hidden state without modifications
        return output, hidden

    def initHidden(self):
        # For bidirectional LSTM with 1 layer, first dimension is 2
        h0 = torch.zeros(2, 1, self.hidden_size, device=device)
        c0 = torch.zeros(2, 1, self.hidden_size, device=device)
        return (h0, c0)

# Initialize the models
hidden_size = 256
encoder_bilstm = EncoderBiLSTM(input_lang.n_words, hidden_size).to(device)
decoder_gru = Decoder(hidden_size, output_lang.n_words).to(device)

Train the model

In [99]:
# Train the models
trainIters(encoder_bilstm, decoder_gru, 5, print_every=5000)

Epoch: 0/5
1m 38s (- 32m 5s) (5000 4%) 3.4016
3m 21s (- 31m 18s) (10000 9%) 2.9345
5m 3s (- 29m 44s) (15000 14%) 2.7165
6m 48s (- 28m 14s) (20000 19%) 2.5522
Epoch: 1/5
8m 30s (- 26m 35s) (25000 24%) 2.3439
10m 13s (- 24m 54s) (30000 29%) 2.2036
11m 57s (- 23m 15s) (35000 33%) 2.0774
13m 40s (- 21m 33s) (40000 38%) 1.9698
Epoch: 2/5
15m 23s (- 19m 51s) (45000 43%) 1.8774
17m 5s (- 18m 8s) (50000 48%) 1.7952
18m 48s (- 16m 26s) (55000 53%) 1.6908
20m 32s (- 14m 44s) (60000 58%) 1.6337
Epoch: 3/5
22m 15s (- 13m 2s) (65000 63%) 1.5699
23m 57s (- 11m 19s) (70000 67%) 1.5030
25m 40s (- 9m 36s) (75000 72%) 1.4556
27m 23s (- 7m 54s) (80000 77%) 1.3915
Epoch: 4/5
29m 6s (- 6m 11s) (85000 82%) 1.3258
30m 50s (- 4m 28s) (90000 87%) 1.2931
32m 33s (- 2m 46s) (95000 92%) 1.2397
34m 16s (- 1m 3s) (100000 97%) 1.1826


Save the model

In [104]:
# Save the models
torch.save(encoder_bilstm.state_dict(), 'encoder_bilstm.pth')
torch.save(decoder_gru.state_dict(), 'decoder_gru.pth')

Load t he models

In [101]:
# Load the models based on the device type
if device.type == 'cuda':
    # Load directly to GPU
    encoder_bilstm.load_state_dict(torch.load('encoder_bilstm.pth'))
    decoder_gru.load_state_dict(torch.load('decoder_gru.pth'))
else:
    # Load to CPU with map_location parameter
    encoder_bilstm.load_state_dict(torch.load('encoder_bilstm.pth', map_location=device))
    decoder_gru.load_state_dict(torch.load('decoder_gru.pth', map_location=device))

<All keys matched successfully>

Rewrite the evaluate function

In [106]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        # Adjust encoder outputs size based on encoder type
        output_size = encoder.hidden_size
        if isinstance(encoder, EncoderBiLSTM):
            output_size = encoder.hidden_size * 2  # Double size for bidirectional

        # FIX: Use square brackets for tensor dimensions
        encoder_outputs = torch.zeros([max_length, output_size], device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

            # Handle different encoder types
            if isinstance(encoder, EncoderBiLSTM):
                encoder_outputs[ei] = encoder_output[0]
            else:
                encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        # If using BiLSTM, prepare hidden state for the decoder
        if isinstance(encoder, EncoderBiLSTM):
            h = encoder_hidden[0][0].unsqueeze(0)  # Take first direction's hidden state
            decoder_hidden = h
        else:
            decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

Evaluate randomly

In [107]:
# Evaluate randomly
evaluateRandomly(encoder_bilstm, decoder_gru)

> tu te leurres .
= you are deceiving yourself .
< you are looking yourself . <EOS>

> tu es une bonne etudiante .
= you re a good student .
< you re a good student . <EOS>

> elle m a prete son velo .
= she s lent me her bicycle .
< she s her me her new . <EOS>

> je me suis assure que personne ne regardait .
= i made sure that no one was watching .
< i m sure that he can t <EOS>

> c est mon dernier avertissement !
= i m warning you for the last time !
< i m the last one . . <EOS>

> il est fort malade .
= he s very sick .
< he s very ill . <EOS>

> je fais mes devoirs .
= i m doing my homework .
< i m doing my homework . <EOS>

> il n a pas encore ecrit la lettre .
= he still has not written the letter .
< he still not always about it . <EOS>

> il est tres beau .
= he is very handsome .
< he s very poor . <EOS>

> je l ai fait partir .
= i made him go .
< i made it . . <EOS>



Test with test pairs and record Rouge sccores

In [108]:
# Test with test pairs and record Rouge scores
input_bilstm, gt_bilstm, predict_bilstm, score_bilstm = test(encoder_bilstm, decoder_gru, test_pairs)

100%|██████████| 2291/2291 [04:03<00:00,  9.41it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.61796856
Rouge1 precision:	 0.58273584
Rouge1 recall:  	 0.6665224
Rouge2 fmeasure:	 0.4351948
Rouge2 precision:	 0.40390694
Rouge2 recall:  	 0.48045906


Task 4: Add Attention Mechanism between Encoder and Decoder
=======================
Add the attention mechanism between Encoder and Decoder in the original code base, run the code for training, and record the Rouge scores for test set

In [109]:
class AttentionDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=MAX_LENGTH):
        super(AttentionDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length

        self.embedding = nn.Embedding(output_size, hidden_size)
        # Add attention layers
        self.attn = nn.Linear(hidden_size * 2, max_length)
        self.attn_combine = nn.Linear(hidden_size * 2, hidden_size)
        # GRU takes input of combined embedding and attention context
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs):
        # Input shape: [1] (just a single token index)
        # Hidden shape: [1, 1, hidden_size]
        # Encoder_outputs shape: [max_length, hidden_size]

        # 1. Embed the input token
        embedded = self.embedding(input).view(1, 1, -1)  # [1, 1, hidden_size]
        embedded = F.relu(embedded)

        # 2. Calculate attention weights
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)  # [1, max_length]

        # 3. Apply attention weights to encoder outputs to get context vector
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))  # [1, 1, hidden_size]

        # 4. Combine embedded input and attention context vector
        output = torch.cat((embedded[0], attn_applied[0]), 1)  # [1, hidden_size*2]
        output = self.attn_combine(output).unsqueeze(0)  # [1, 1, hidden_size]
        output = F.relu(output)

        # 5. Run the GRU for one step
        output, hidden = self.gru(output, hidden)  # output: [1, 1, hidden_size]

        # 6. Transform GRU output to vocabulary size and apply log softmax
        output = self.softmax(self.out(output[0]))  # [1, output_size]

        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Modified evaluate function for attention decoder
def evaluate_attention(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, attn_weights = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

# Modified train function for attention decoder
def train_attention(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
                    decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

# Training function for attention model
def trainIters_attention(encoder, decoder, epochs, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    iter = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print(f"Epoch: {epoch+1}/{epochs}")
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)

            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train_attention(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))
            iter += 1

# Function to evaluate the model randomly
def evaluateRandomly_attention(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate_attention(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# Function to test the model and record ROUGE scores
def test_attention(encoder, decoder, testing_pairs):
    input = []
    gt = []
    predict = []
    metric_score = {
        "rouge1_fmeasure":[],
        "rouge1_precision":[],
        "rouge1_recall":[],
        "rouge2_fmeasure":[],
        "rouge2_precision":[],
        "rouge2_recall":[]
    }
    from tqdm import tqdm
    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate_attention(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)

        input.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

        try:
            rs = rouge(output_sentence, pair[1])
        except:
            continue
        metric_score["rouge1_fmeasure"].append(rs['rouge1_fmeasure'])
        metric_score["rouge1_precision"].append(rs['rouge1_precision'])
        metric_score["rouge1_recall"].append(rs['rouge1_recall'])
        metric_score["rouge2_fmeasure"].append(rs['rouge2_fmeasure'])
        metric_score["rouge2_precision"].append(rs['rouge2_precision'])
        metric_score["rouge2_recall"].append(rs['rouge2_recall'])

    metric_score["rouge1_fmeasure"] = np.array(metric_score["rouge1_fmeasure"]).mean()
    metric_score["rouge1_precision"] = np.array(metric_score["rouge1_precision"]).mean()
    metric_score["rouge1_recall"] = np.array(metric_score["rouge1_recall"]).mean()
    metric_score["rouge2_fmeasure"] = np.array(metric_score["rouge2_fmeasure"]).mean()
    metric_score["rouge2_precision"] = np.array(metric_score["rouge2_precision"]).mean()
    metric_score["rouge2_recall"] = np.array(metric_score["rouge2_recall"]).mean()

    print("=== Evaluation score - Rouge score ===")
    print("Rouge1 fmeasure:\t",metric_score["rouge1_fmeasure"])
    print("Rouge1 precision:\t",metric_score["rouge1_precision"])
    print("Rouge1 recall:  \t",metric_score["rouge1_recall"])
    print("Rouge2 fmeasure:\t",metric_score["rouge2_fmeasure"])
    print("Rouge2 precision:\t",metric_score["rouge2_precision"])
    print("Rouge2 recall:  \t",metric_score["rouge2_recall"])
    print("=====================================")
    return input,gt,predict,metric_score

# Initialize the models
hidden_size = 256
encoder_attn = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_attn = AttentionDecoder(hidden_size, output_lang.n_words).to(device)

Train the model

In [110]:
trainIters_attention(encoder_attn, decoder_attn, 5, print_every=5000)

Epoch: 1/5
1m 56s (- 38m 7s) (5000 4%) 3.2231
3m 55s (- 36m 32s) (10000 9%) 2.7234
5m 54s (- 34m 42s) (15000 14%) 2.4826
7m 54s (- 32m 49s) (20000 19%) 2.3230
Epoch: 2/5
9m 53s (- 30m 52s) (25000 24%) 2.1728
11m 52s (- 28m 56s) (30000 29%) 2.0256
13m 54s (- 27m 2s) (35000 33%) 1.9168
15m 55s (- 25m 7s) (40000 38%) 1.8526
Epoch: 3/5
17m 59s (- 23m 12s) (45000 43%) 1.7791
20m 0s (- 21m 13s) (50000 48%) 1.6852
22m 1s (- 19m 14s) (55000 53%) 1.5918
24m 3s (- 17m 16s) (60000 58%) 1.5698
Epoch: 4/5
26m 5s (- 15m 17s) (65000 63%) 1.5094
28m 9s (- 13m 18s) (70000 67%) 1.4695
30m 11s (- 11m 18s) (75000 72%) 1.3977
32m 13s (- 9m 17s) (80000 77%) 1.4019
Epoch: 5/5
34m 16s (- 7m 17s) (85000 82%) 1.3575
36m 19s (- 5m 16s) (90000 87%) 1.3308
38m 20s (- 3m 15s) (95000 92%) 1.3033
40m 21s (- 1m 14s) (100000 97%) 1.2814


Save the models

In [111]:
torch.save(encoder_attn.state_dict(), 'encoder_attn.pth')
torch.save(decoder_attn.state_dict(), 'decoder_attn.pth')

Load the models

In [ ]:
# Load the models based on the device type
if device.type == 'cuda':
    # Load directly to GPU
    encoder_attn.load_state_dict(torch.load('encoder_attn.pth'))
    decoder_attn.load_state_dict(torch.load('decoder_attn.pth'))
else:
    # Load to CPU with map_location parameter
    encoder_attn.load_state_dict(torch.load('encoder_attn.pth', map_location=device))
    decoder_attn.load_state_dict(torch.load('decoder_attn.pth', map_location=device))

Evaluate the models randomly

In [112]:
evaluateRandomly_attention(encoder_attn, decoder_attn)

> il va leur montrer les documents .
= he s going to show them the documents .
< he s going to show the the <EOS>

> tu es la personne que j attendais .
= you re the person i ve been waiting for .
< you re the person that i ve been waiting for <EOS>

> tu ne vas pas le faire n est ce pas ?
= you re not going to do it are you ?
< you aren t want to do that are <EOS>

> tu as tout a fait raison .
= you re exactly right .
< you re exactly right . <EOS>

> tu as toujours a te plaindre .
= you are always complaining .
< you re always complaining . . <EOS>

> il ne craint personne .
= he s not afraid of anyone .
< he is not afraid . <EOS>

> elles vont te trouver .
= they re going to find you .
< they re going to find you . <EOS>

> je n ai pas peur de la mort .
= i m not afraid of death .
< i m not afraid of death . <EOS>

> je vais repasser un coup de wassingue .
= i m going to mop this up .
< i m going to watch a book <EOS>

> je ne suis pas si mauvais .
= i m not that bad .
< i m not tha

Test with test pairs and record Rouge scores

In [113]:
input_attn, gt_attn, predict_attn, score_attn = test_attention(encoder_attn, decoder_attn, test_pairs)

100%|██████████| 2291/2291 [11:23<00:00,  3.35it/s]

=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.59745514
Rouge1 precision:	 0.5673051
Rouge1 recall:  	 0.64152175
Rouge2 fmeasure:	 0.4083379
Rouge2 precision:	 0.38092256
Rouge2 recall:  	 0.45012605


Task 5: Replace Encoder GRU with Transformer Encoder
=======================
Change the GRU in Encoder (not Decoder) in the original code base with Transformer Encoder, run the code, and record the Rouge scores for test set

In [114]:
import math

# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

# Transformer Encoder
class EncoderTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, nhead=8, num_layers=2, dropout=0.1):
        super(EncoderTransformer, self).__init__()
        self.hidden_size = hidden_size

        # Ensure hidden_size is divisible by nhead
        if hidden_size % nhead != 0:
            adjusted_nhead = hidden_size // (hidden_size // nhead)
            print(f"Warning: hidden_size ({hidden_size}) not divisible by nhead ({nhead}). Adjusting nhead to {adjusted_nhead}.")
            nhead = adjusted_nhead

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size, dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=nhead,
            dim_feedforward=hidden_size*4,
            dropout=dropout,
            batch_first=True
        )

        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

    def forward(self, input, hidden):
        # Embed input
        embedded = self.embedding(input).view(1, 1, -1)  # [1, 1, hidden_size]

        # Add positional encoding
        embedded = self.pos_encoder(embedded)

        # Pass through transformer encoder
        transformer_output = self.transformer_encoder(embedded)  # [1, 1, hidden_size]

        # For GRU compatibility
        return transformer_output, transformer_output

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Initialize the models
hidden_size = 256
# For transformer, we need to ensure hidden_size is divisible by nhead
nhead = 8  # Number of attention heads
num_layers = 2  # Number of transformer encoder layers
dropout = 0.1  # Dropout probability

encoder_transformer = EncoderTransformer(input_lang.n_words, hidden_size, nhead, num_layers, dropout).to(device)
decoder_gru = Decoder(hidden_size, output_lang.n_words).to(device)

Train the models

In [115]:
trainIters(encoder_transformer, decoder_gru, 5, print_every=5000)

Epoch: 0/5
2m 11s (- 43m 2s) (5000 4%) 3.5155
4m 23s (- 40m 51s) (10000 9%) 3.2547
6m 37s (- 38m 52s) (15000 14%) 3.1818
8m 49s (- 36m 41s) (20000 19%) 3.1449
Epoch: 1/5
11m 2s (- 34m 29s) (25000 24%) 3.0823
13m 15s (- 32m 16s) (30000 29%) 3.0238
15m 25s (- 30m 1s) (35000 33%) 3.0138
17m 35s (- 27m 45s) (40000 38%) 3.0015
Epoch: 2/5
19m 45s (- 25m 29s) (45000 43%) 2.9659
21m 54s (- 23m 15s) (50000 48%) 2.9306
24m 4s (- 21m 2s) (55000 53%) 2.9284
26m 13s (- 18m 49s) (60000 58%) 2.8976
Epoch: 3/5
28m 23s (- 16m 37s) (65000 63%) 2.8756
30m 31s (- 14m 25s) (70000 67%) 2.8779
32m 39s (- 12m 13s) (75000 72%) 2.8427
34m 47s (- 10m 2s) (80000 77%) 2.8489
Epoch: 4/5
37m 0s (- 7m 52s) (85000 82%) 2.8135
39m 12s (- 5m 41s) (90000 87%) 2.8050
41m 22s (- 3m 31s) (95000 92%) 2.7775
43m 31s (- 1m 20s) (100000 97%) 2.7790


Save the models

In [116]:
torch.save(encoder_transformer.state_dict(), 'encoder_transformer.pth')
torch.save(decoder_gru.state_dict(), 'decoder_gru.pth')

Load the models

In [119]:
# Load the models based on the device type
if device.type == 'cuda':
    # Load directly to GPU
    encoder_transformer.load_state_dict(torch.load('encoder_transformer.pth'))
    decoder_gru.load_state_dict(torch.load('decoder_gru.pth'))
else:
    # Load to CPU with map_location parameter
    encoder_transformer.load_state_dict(torch.load('encoder_transformer.pth', map_location=device))
    decoder_gru.load_state_dict(torch.load('decoder_gru.pth', map_location=device))

Evaluate the models

In [120]:
evaluateRandomly(encoder_transformer, decoder_gru)

> je suis enferme hors de ma chambre .
= i m locked out of my room .
< i m not . . . . <EOS>

> je suis heureux que tu l aies apprecie .
= i m glad you liked it .
< i m not . . . . <EOS>

> je suis desolee de vous ennuyer si souvent .
= i m sorry to bother you so often .
< i m not . . . . <EOS>

> je garde cette place pour un ami .
= i m saving this seat for a friend .
< i m not . . . . <EOS>

> il lui a vole un baiser .
= he stole a kiss from her .
< i m not . . . . <EOS>

> il est incapable de mentir .
= he is incapable of telling a lie .
< i m not . . . . <EOS>

> je suis au regime en ce moment .
= i m on a diet at the moment .
< i m not . . . . <EOS>

> vous vous trouvez sur la liste .
= you re on the list .
< i m not . . . . <EOS>

> je retourne a mon bureau .
= i m going back to my office .
< i m not . . . . <EOS>

> elle lui serra la main .
= she shook hands with him .
< i m not . . . . <EOS>



Test with test pairs and record Rouge scores

In [121]:
input_transformer, gt_transformer, predict_transformer, score_transformer = test(encoder_transformer, decoder_gru, test_pairs)

100%|██████████| 2291/2291 [18:54<00:00,  2.02it/s]

=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.20694938
Rouge1 precision:	 0.26038846
Rouge1 recall:  	 0.17930742
Rouge2 fmeasure:	 0.12091199
Rouge2 precision:	 0.16142878
Rouge2 recall:  	 0.10299744
